In [2]:
def simpleReactionOne(fileName,modelName,*connections):
    import BondGraphTools as bgt

    [modelNum, subName, elements, init_vals]=getCellOne(fileName)
    [reaction,B,zero_junc_a,zero_junc_b , qB_value, R_value, T_value,r_value]=bgCompReactionOne(modelNum, elements,init_vals)
    [connectivity_bonds_column,connectivity_bonds_row]=CBR_reactionOneSorting(reaction,zero_junc_a,zero_junc_b,B)
    
    model=bgt.new(name=modelName)
    bgt.add(model,reaction,zero_junc_a,zero_junc_b,B)
    
    [Con_Mod,Port_Mod]=Index_C_One(subName,*connections)
    connectivity_matrix_0=CM_CreationReactionOne(modelNum)
    connectivity_matrix=Whole_CM_One(Con_Mod,Port_Mod,connectivity_bonds_row,connectivity_matrix_0)
    bonds=BondsConnectionOne(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column)
    [model,qA_value]=Get_subsReaction(modelNum,subName,bonds,model,R_value, T_value)
    x_0=combineDictOne(qB_value,qA_value)
    
    return [x_0,model,r_value]
    

In [3]:
def getCellOne(fileName):
    import requests
    import xml.etree.ElementTree as ET
    import pprint
    import numpy as np
    from copy import copy, deepcopy
    
    f = open(fileName,'r')
    text = f.read()
    root = ET.fromstring(text)
    components = root.findall('{http://www.cellml.org/cellml/1.1#}component')
    
    # Getting the names of all the sub-systems ==> subName
    subName=[]
    for c in components:
        subName.append(c.get('name'))
    
    #number of models in the file 
    modelNum=len(components)
    #number of variables in each model 
    compNum=len(components[0])
    
    
    variables=[]; els=[]; init=[]
    
    for comp in components:
        variables.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
 
    for var in variables:
        for v in var:
            els.append(v.attrib['name'])
            if 'initial_value' in v.attrib: # if any initial value exists take it
                init.append(v.attrib['initial_value'])    

    # Separating the groups of elements in each model
    j=0
    elements=[]
    for i in range(0,modelNum*compNum,compNum):    
        elements.append(els[i:i+compNum].copy())
        j+=1
    # Separating the groups of initial values in each model
    j=0
    init_vals=[]
    for i in range(0,modelNum*compNum,compNum):    
        init_vals.append(init[i:i+compNum].copy())
        j+=1
        
    return [modelNum,subName, elements, init_vals]

[modelNum,subName, elements, init_vals]=getCellOne('Reactions.cellml')

In [4]:
def bgCompReactionOne(modelNum, elements,init_vals):
    
    import numpy as np
    import BondGraphTools as bgt
    
    # assigning the value of each element to its equivalent component value in bond graphs
    qA_value=[]; qB_value=[]; 
    K_A_value=[]; K_B_value=[];
    r_value=[]; R_value=[]; T_value=[];  
    
    for m in range(modelNum):
        for k in range(len(elements[1])):
#             if elements[m][k]=='qA':
#                 qA_value.append(init_vals[m][k])
            if elements[m][k]=='qB':
                qB_value.append(float(init_vals[m][k]))
#             elif elements[m][k]=='K_A':
#                 K_A_value.append(init_vals[m][k])
            elif elements[m][k]=='K_B':
                K_B_value.append(float(init_vals[m][k]))
            elif elements[m][k]=='r':
                r_value.append(float(init_vals[m][k]))
            elif elements[m][k]=='R':
                R_value.append(float(init_vals[m][k]))
            elif elements[m][k]=='T':
                T_value.append(float(init_vals[m][k]))
            
    # Here by each iteration we create the BG components of each model ...
    # by adding the model number at the end of each component name, indicating that ...
    # each component belongs to which model
    reaction=[];
    B=[]; 
    zero_junc_a=[]; zero_junc_b=[]; 
    
    for m in range(modelNum): 
        B.append(bgt.new("Ce", library="BioChem", value={'k':float(K_B_value[m]), 'R':float(R_value[m]), 'T':float(T_value[m])}, name='B_'+str(m)))

        zero_junc_a.append(bgt.new("0", name='zero_junc_a_'+str(m)))
        zero_junc_b.append(bgt.new("0", name='zero_junc_b_'+str(m)))

        reaction.append(bgt.new("Re", library="BioChem", value={'r':None, 'R':float(R_value[m]), 'T':float(T_value[m])}, name='Reaction_'+str(m)))
    return [reaction,B,zero_junc_a,zero_junc_b , qB_value, R_value, T_value,r_value]

    

[reaction,B,zero_junc_a,zero_junc_b , qB_value, R_value, T_value,r_value]=bgCompReactionOne(modelNum, elements,init_vals)

In [5]:
def CBR_reactionOneSorting(reaction,zero_junc_a,zero_junc_b,B):
    # By the following appending, all the same components in each model are put together
    # So we need to separate them and gather all the components of each model in a row
    CBR=[]
    CBR.append([reaction,zero_junc_a,zero_junc_b,B])  
    
    connectivity_bonds_row=[]
    for j in range(len(CBR[0][0])):
        for i in range(len(CBR[0])):
            connectivity_bonds_row.append(CBR[0][i][j])
            
    connectivity_bonds_column=connectivity_bonds_row
            
    return [connectivity_bonds_column,connectivity_bonds_row]        

[connectivity_bonds_column,connectivity_bonds_row]=CBR_reactionTwoSorting(zero_junc_s,one_junc_s,reaction,one_junc_p,zero_junc_p,P)

In [6]:
def Index_C_One(subName,*con):    
    
    connection=list(con)
    conIter=int(len(connection)/4)
    connecting_models=[]
    ports=[]
    ConMod=[]
   # Dividing the elements in the input connections into lists with 4 elements, so each list represents ...
   # a separate connection between the modules
    for p in range(0,len(connection),4):
        ConMod.append(connection[p:p+4])
    
    # For the given inputs in connections, we assign numbers so the calculations can go on ...
    # For instance we assign {0,1} for {'in','out'} respectively and we assign {0,1,2,...} for the model components
    for m in range(conIter):
        for i in range(0,4):
            for j in range(len(subName)):
                if ConMod[m][i]==subName[j]:
                    connecting_models.append(j)
                    if ConMod[m][i+1]=='in':
                        ports.append(0)
                    if ConMod[m][i+1]=='out':
                        ports.append(1)


    
    # Now we split the connection modules and ports two by two to specify the detailed connections:
    Con_Mod=[]
    Port_Mod=[]
    for p in range(0,len(connecting_models),2):
        Con_Mod.append(connecting_models[p:p+2])
        Port_Mod.append(ports[p:p+2])

    
    
    return [Con_Mod,Port_Mod]

[Con_Mod,Port_Mod]=Index_C(subName,'reaction_1','out','reaction_2','in',      'reaction_2','out','reaction_3','in')

In [7]:
def CM_CreationReactionOne(modelNum):

    import numpy as np
    from copy import copy, deepcopy
    
     #  connectivity matrix for one reaction with one input/output 
    connectivity_matrix_D=[[0,0,1,0],
                           [1,0,0,0],
                           [0,0,0,1],
                           [0,0,0,0]]


    CM={}
    
    # Creating a dictionary with empty lists for adding the connectivity matrices for each sub-system
    for n in range(modelNum):
        CM[str(n)]=deepcopy(connectivity_matrix_D)
    

    Length=0
    for n in range(modelNum):
        Length+=len(CM[str(n)])
    connectivity_matrix_0=np.zeros((Length,Length))  

    
    k=0
    for n in range(modelNum):
        for i in range(k,k+len(CM[str(n)])):
            for j in range(k,k+len(CM[str(n)])):
                connectivity_matrix_0[i][j]=deepcopy(CM[str(n)][i-k][j-k])
        k+=len(CM[str(n)])
        
    return connectivity_matrix_0

connectivity_matrix_0=CM_CreationReactionTwo(modelNum,zero_junc_s,zero_junc_p,P)

In [8]:
def Whole_CM_One(Con_Mod,Port_Mod,connectivity_bonds_row,connectivity_matrix_0):
    from copy import copy, deepcopy
    Name=[]
    for i in range(len(Con_Mod)):
        for j in range(2):
            m=Con_Mod[i][j]
            p=Port_Mod[i][j]
            if p == 0:
                Name.append('zero_junc_a_'+str(m))
            elif p == 1:
                Name.append('zero_junc_b_'+str(m))
           
    Connection_between_models=[]
    for name in Name:
        for index in range(len(connectivity_bonds_row)):
            if name == connectivity_bonds_row[index].name:
                Connection_between_models.append(index)
                break
     
    
    connectivity_matrix=deepcopy(connectivity_matrix_0)
    for i in range(0,len(Connection_between_models),2):             
            II=Connection_between_models[i]
            JJ=Connection_between_models[i+1]
            connectivity_matrix[II][JJ]=1
        
    return connectivity_matrix

connectivity_matrix=Whole_CM_Two(Con_Mod,Port_Mod,zero_junc_s,zero_junc_p,connectivity_bonds_row,connectivity_matrix_0)

In [9]:
def BondsConnectionOne(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column):
    import numpy as np
    import BondGraphTools as bgt
# Creating the bonds between the components based on the elements of the connectivity matrix
    Length=len(connectivity_matrix)
    bonds=[]  
    for i in range(Length):
        for j in range(Length):
            if connectivity_matrix[i][j]==1:
                bonds.append((connectivity_bonds_row[i],connectivity_bonds_column[j]))
            else:
                m=0
    
    
    # connecting the components for a bond graph representation of the system
    for head, tail in bonds:
        bgt.connect(head,tail)
    
    return bonds

bonds=BondsConnection(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column)

In [10]:
def Get_subsReaction(modelNum,subName,bonds,model,R_value, T_value):
    import BondGraphTools as bgt


#     for m in range(0,modelNum):
    itr_a=0

    for i in range(0,len(bonds)):
        for j in range(0,2):
            if bonds[i][j].name=='zero_junc_a_0':
                itr_a+=1
    count_zero_junc_a=itr_a


    boundary_names = ['K_A','qA']
    boundary_values = {}
    A=[]; 

#     for m in range(0,modelNum):
    m=0
    if count_zero_junc_a < 2:
        boundary_values[boundary_names[0]+'_'+str(m)+'_value']=input('Enter value for {} in substrate "A" of model {}: '.format(boundary_names[0],subName[m]))
        boundary_values[boundary_names[1]+'_'+str(m)+'_value']=input('Enter initial value for the state variable {} in substrate "A" of model {}: '.format(boundary_names[1],subName[m]))
        A.append(bgt.new("Ce",library="BioChem", value={'k':float(boundary_values[boundary_names[0]+'_'+str(m)+'_value']), 'R':float(R_value[m]), 'T':float(T_value[m])}, name='A_'+str(m)))

    bgt.add(model,A)
    
    m=0
    for index in range(len(model.components)):
#         for m in range(1):
        if 'A_'+str(m) == model.components[index].name:
            for j in range(len(model.components)):
                if 'zero_junc_a_'+str(m) == model.components[j].name:
                    bgt.connect(model.components[index],model.components[j])

  
    return [model,boundary_values[boundary_names[1]+'_'+str(0)+'_value']]

In [11]:
def combineDictOne(*dicts):
    incKey = 0
    newDict = {}
    for dictionary in dicts:
        for val in dictionary:
            newDict['x_'+str(incKey)] = val 
            incKey+=1
    return newDict